# Загрузка базы вопросов ЧГК

Нужно загрузить все доступные вопросы и ответы к ним с базы http:///db.chgk.info . Результаты необходимо сохранить в каком-либо формате в форме вопрос-ответ (pickle, sqlite, текстовый файл и т.д.). 

In [1]:
import lxml.html
import requests
import re

In [2]:
page = requests.get('http://db.chgk.info/')
tree_page = lxml.html.fromstring(page.text)

In [3]:
tree_page.xpath('//title/text()')

['База вопросов "Что? Где? Когда?".']

In [4]:
pattern = re.compile(r'\d+')
number_of_pages = pattern.findall(tree_page.xpath('//li[contains(@class, "pager-last last")]/a/@href')[0])[0]

In [5]:
links = []
for i in range(int(number_of_pages)):
    response = requests.get('http://db.chgk.info/last?page=' + str(i))
    tree = lxml.html.fromstring(response.text)
    p = tree.xpath('//table[contains(@class, "last_packages")]//td[2]/a/@href')
    links.extend(list(map(lambda a: 'http://db.chgk.info/' + a, p)))

In [6]:
with open('links.txt', 'w') as f:
    f.write('\n'.join(links))

In [7]:
def function(question):
    elements = question.xpath('.//p/strong')[:2]
    return 'Q: {}\nA: {}\n'.format(*map(lambda a: a.tail.strip(), elements))


In [8]:
i=0
chgk_base = []    
with open('links.txt', 'r') as file:
    for line in file:
        i += 1
        if i == 10:
            break
        response = requests.get(line.strip())
        tree = lxml.html.fromstring(response.text)
        p = tree.xpath('//div[@class = "question"]')
        chgk_base.extend(list(map(function, p)))
    
with open('chgk.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(chgk_base))
 
    